In [4]:
from banners import LocalBanner

def callback_func(d: dict):
    print("IN CALLBACK", d)

banner = LocalBanner(watch_rate=1)


In [5]:
banner.watch("test", callback_func)

IN CALLBACK {'test': 'value', 'topic': 'test'}
IN CALLBACK {'test': 'value', 'topic': 'test'}


In [8]:
banner.wave("test", {"test": "value"})


In [7]:
banner.ignore("test")


In [9]:
banner.retire("test", 0)